<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1_Basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import OneHotEncoder

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


# Setup Weights and Bias.

* AI experimental tool.

In [29]:
!pip install wandb -qU

In [30]:
# Log in to your W&B account
import wandb
import random
import math

In [31]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Make RNN

In [32]:
class RNN(nn.Module):
    def __init__(self,max_laps,hidden_size):
        super(RNN, self).__init__()
        self.embedding =  nn.Embedding(max_laps, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,max_laps)
        self.tahn = nn.Tanh()

    def forward(self,x,hprev):
        h = self.tahn(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

# Loss function (Based upon wk11 practical)

* Vocab size is not relevent here - going to give it binary classification values anyway.
* Everything else can be applied for time series: use Cross Entropy Loss, Length of the race = seq_len.
* For each lap, pass through the RNN with previous hidden state and build up hte matrix of outputs.
* Detact the previous hidden state as a new one will be generated.

In [ ]:
def calculate_loss(model,inputs,targets,hprev,max_tyre_life):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  #outputs = torch.zeros(0, max_laps)
  #print(outputs.shape)
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(inputs[t], hprev)
    # Gradually build up matrix of output logits of size seq_length * max_tyre_life
    #outputs = torch.cat((outputs ,y))

  y = y[-1]

  # Compute cross entropy loss for seq_length actual targets against estimated distributions
  loss = loss_func(y,targets.squeeze(0))

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

# My dataset - how to format?

* Training and testing
* inputs_training/ inputs_testing/  outputs_testing/ output_training
* For now, it will all just be a 2d array.

ERROR:

```2551     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2552
   2553

IndexError: index out of range in self```

In [ ]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

In [ ]:
# Define compound categories and one-hot encode them
compound_categories = ['SOFT', 'MEDIUM', 'HARD', 'WET', 'INTERMEDIATE']
compound_encoder = OneHotEncoder(categories=[compound_categories], sparse_output=False)

In [ ]:
# Define the logical ordering of compounds
compound_order = {'SOFT': 1, 'MEDIUM': 2, 'HARD': 3, 'INTERMEDIATE': 4, 'WET': 5}

# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            """if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])"""

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values

            # Map compounds to ordinal values
            compound_encoded = np.array([compound_order[compound] for compound in compound_array])

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long, device=device)
            compound_encoded = torch.tensor(compound_encoded, dtype=torch.long, device=device).unsqueeze(1)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32, device=device)

            combined_inputs = torch.cat([tyre_life_array.unsqueeze(1), compound_encoded], dim=1)

            # Add to training data
            training_inputs.append(combined_inputs)
            training_outputs.append(stint_change_array)

In [ ]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values

            # Map compounds to ordinal values
            compound_encoded = np.array([compound_order[compound] for compound in compound_array])

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long, device=device)
            compound_encoded = torch.tensor(compound_encoded, dtype=torch.long, device=device).unsqueeze(1)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32, device=device)

            combined_inputs = torch.cat([tyre_life_array.unsqueeze(1), compound_encoded], dim=1)

            # Add to testing data
            testing_inputs.append(combined_inputs)
            testing_outputs.append(stint_change_array)

## Modify code for sliding window approach.

* Look at the full race image instead.

In [ ]:
def create_sliding_windows_pandas(data, targets, window_size, step_size):
    windows = []
    # Convert to DataFrame
    data_df = pd.DataFrame(data)
    targets_df = pd.DataFrame(targets)

    # Iterate to create windows
    for start in range(0, len(data_df) - window_size + 1, step_size):
        end = start + window_size
        # Extract window for features and targets
        input_window = data_df.iloc[start:end].values
        target_window = targets_df.iloc[start:end].values
        windows.append((torch.tensor(input_window, dtype=torch.long), torch.tensor(target_window, dtype=torch.float32)))

    return windows

# Example: Apply this function within your loop for each driver
training_inputs = []
training_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
# Assuming you've loaded data and processed it as you did before:
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Extract features as before
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values
            compound_encoded = np.array([compound_order[compound] for compound in compound_array])

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:
                    stint_change_array.append(False)
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            # Convert arrays to pandas Series for rolling windows
            input_data = np.column_stack([tyre_life_array, compound_encoded])
            output_data = np.array(stint_change_array)

            # Apply sliding window function
            windows = create_sliding_windows_pandas(input_data, output_data, window_size=10, step_size=5)

            # Add to training data
            for input_window, output_window in windows:
                training_inputs.append(input_window)
                training_outputs.append(output_window)


In [ ]:
import torch
import numpy as np
import pandas as pd

def create_sliding_windows_pandas(data, targets, window_size, step_size):
    windows = []
    # Convert to DataFrame
    data_df = pd.DataFrame(data)
    targets_df = pd.DataFrame(targets)

    # Iterate to create windows
    for start in range(0, len(data_df) - window_size + 1, step_size):
        end = start + window_size
        # Extract window for features and targets
        input_window = data_df.iloc[start:end].values
        target_window = targets_df.iloc[start:end].values
        windows.append((torch.tensor(input_window, dtype=torch.long), torch.tensor(target_window, dtype=torch.float32)))

    return windows

# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values

            # Map compounds to ordinal values
            compound_encoded = np.array([compound_order[compound] for compound in compound_array])

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Prepare data for sliding windows
            input_data = np.column_stack([tyre_life_array, compound_encoded])
            output_data = np.array(stint_change_array)

            # Apply sliding window function
            windows = create_sliding_windows_pandas(input_data, output_data, window_size=10, step_size=5)

            # Add windows to testing data
            for input_window, output_window in windows:
                testing_inputs.append(input_window)
                testing_outputs.append(output_window)

# Training the model

* https://developers.google.com/machine-learning/crash-course/overfitting/interpreting-loss-curves

In [ ]:
hidden_size = 10
lr = 0.01
iterations = 10000
max_race_size = int(max_race_size)
input_parameters = ['TyreLife', 'Compound']
dataset = 'v1'

lr_experiments = [0.01, 0.05, 0.001, 0.005]

In [ ]:
def evaluate_model(inputs, outputs, tolerance=3):
    total_correct = 0
    total_predictions = 0

    for seq_idx in range(len(inputs)):
        # Initialize hidden state for each sequence
        hprev = torch.zeros(hidden_size, device=device)

        # Get the input and target sequences
        input = inputs[seq_idx]
        output = outputs[seq_idx].cpu().numpy()

        # Step through the input sequence and get predicted probabilities
        for t in range(len(input)):
            hprev, y = model(torch.tensor(input[t], device=device), hprev)
            predicted_pits = torch.sigmoid(y) # Apply sigmoid to logits to get probabilities

        y = y[-1]

        predicted_pits = predicted_pits.cpu().detach().numpy()[0]
        predicted_pits = predicted_pits > 0.5
        actual_pits = output == 1.0

        # Now check accuracy for this sequence with tolerance
        correct_count = 0
        total_predictions_for_sequence = 0

        for predicted_pit_idx in range(len(predicted_pits)):
          if predicted_pits[predicted_pit_idx]:  # If a pit stop is predicted
              total_predictions += 1

              # Calculate the tolerance window around the predicted pit
              start_idx = max(0, predicted_pit_idx - tolerance)
              end_idx = min(len(actual_pits), predicted_pit_idx + tolerance + 1)

              # Check if there's any actual pit stop within this window using NumPy
              if np.any(actual_pits[start_idx:end_idx] == 1):  # If there's a pit stop in this window
                  correct_count += 1
                  break

        total_correct += correct_count

    # Calculate overall accuracy
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0
    return accuracy * 100

In [ ]:
def pad_with_last_value(tensor, max_race_size):
    if len(tensor) >= max_race_size:
        return tensor

    last_value = tensor[-1]

    # TO FIX: RuntimeError: a Tensor with 6 elements cannot be converted to Scalar
    # After adding compound
    if last_value.ndimension() > 0:
        padding = last_value.unsqueeze(0).repeat(max_race_size - len(tensor), 1)
    else:
        padding = torch.full((max_race_size - len(tensor),), last_value.item(), dtype=tensor.dtype, device=device)

    return torch.cat((tensor, padding))

In [ ]:
optimizers = {
    "SGD": torch.optim.SGD,  # Stochastic Gradient Descent
    #"RMSprop": torch.optim.RMSprop,  # Root Mean Square Propagation
    #"Adagrad": torch.optim.Adagrad,  # Adaptive Gradient Algorithm
    #"Adadelta": torch.optim.Adadelta,  # Adaptive Delta
    #"AdamW": torch.optim.AdamW,  # Adam with decoupled weight decay
    #"NAdam": torch.optim.NAdam,  # Nesterov-accelerated Adaptive Moment Estimation
    #"ASGD": torch.optim.ASGD,  # Averaged Stochastic Gradient Descent
    #"SparseAdam": torch.optim.SparseAdam,  # Optimizer for sparse tensors
    #"Rprop": torch.optim.Rprop,  # Resilient Backpropagation
}

permutation = torch.randperm(len(training_inputs))
training_inputs = [training_inputs[i] for i in permutation]
training_outputs = [training_outputs[i] for i in permutation]

for lr in lr_experiments:
  model = RNN(max_race_size, hidden_size)
  model = model.to(device)

  optim = optimizers["SGD"](model.parameters(), lr = lr)


  # Initialize WandB
  wandb.init(
      project="prbx-model-v1",
      config={
          "learning_rate": lr,
          "iterations": iterations,
          "hidden_size": hidden_size,
          "max_race_size": max_race_size,
          "input_parameters": input_parameters,
          "dataset": dataset,
          "optimizer": "SGD",
          "random_perms": True,
          "compound_formatting": "logical_ordering",
          "tanh_alternative": "relu",
          "gradient_clipping_method": "fixed 1.0 max norm"
      }
  )

  n, p = 0, 0
  while n<=iterations:
    hprev = torch.zeros(hidden_size,) # reset RNN memory

    inputs = training_inputs[p]
    targets = training_outputs[p]

    inputs = pad_with_last_value(inputs, max_race_size).to(device)
    targets = pad_with_last_value(targets, max_race_size).to(device)
    hprev = torch.zeros(hidden_size, device=device)

    # Compute loss for current subsequence
    loss, hprev = calculate_loss(model,inputs,targets,hprev,max_race_size)
    optim.zero_grad()
    loss.backward()

    # Clip gradients
    max_norm = 1.0  # Maximum norm for gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

    optim.step()

    if n % 500 == 0:
      print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
      testing_accuracy = evaluate_model(testing_inputs,testing_outputs)
      training_accuracy = evaluate_model(training_inputs,training_outputs)

      # Log metrics to WandB
      wandb.log({
          "iteration": n,
          "loss": loss.item(),
          "training_accuracy": training_accuracy,
          "testing_accuracy": testing_accuracy,
      })

      # Save to weights and biases.
      model_path = "model.pth"
      torch.save(model.state_dict(), model_path)


      artifact = wandb.Artifact("basic-rnn-binary-classifier", type="model")
      artifact.add_file(model_path)
      wandb.log_artifact(artifact)

    n += 1 # iteration counter
    p += 1 # dataset counter

    #Restart to the front of the list.
    if len(training_inputs) - 1 == p:
      p = 0
      permutation = torch.randperm(len(training_inputs))
      training_inputs = [training_inputs[i] for i in permutation]
      training_outputs = [training_outputs[i] for i in permutation]

#How can we calculate accuracy for this basic model?

* Say is probability is above 80% percent pit, otherwise don't pit.
* Want at least one right pit stop identification to class it as match.

In [ ]:
for t in range(len(testing_inputs[1])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor(testing_inputs[1][t], device=device),hprev)
probabilities = torch.sigmoid(y)

In [ ]:
probabilities[-1]

In [ ]:
for t in range(len(training_inputs[0])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor(training_inputs[0][t], device=device),hprev)
probabilities = torch.sigmoid(y)

In [ ]:
probabilities[-1]

# Plot to see what this basic model has remembered - probably just averages

# Validating my data

In [ ]:
len(training_inputs) == len(training_outputs)

In [ ]:
training_inputs[0].shape

In [ ]:
training_outputs[0].shape

## Sliding window training here

In [36]:
def calculate_loss(model,inputs,targets,hprev,vocab_size):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  outputs = torch.zeros(0,vocab_size)
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model([inputs[t]],hprev)
    # Gradually build up matrix of output logits of size seq_length * vocab_size
    outputs = torch.cat((outputs,y))

  # Compute cross entropy loss for seq_length actual targets against estimated distributions
  loss = loss_func(outputs,torch.tensor(targets))

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

In [37]:
hidden_size = 10
lr = 0.01
iterations = 10000
max_race_size = int(max_race_size)
input_parameters = ['TyreLife', 'Compound']
dataset = 'v1'

In [39]:
n, p = 0, 0  # Iteration and sliding window pointer
model = RNN(max_race_size, hidden_size)

# Initialize WandB
wandb.init(
    project="prbx-model-v1b",
    config={
        "learning_rate": lr,
        "iterations": iterations,
        "hidden_size": hidden_size,
        "max_race_size": max_race_size,
        "input_parameters": input_parameters,
        "dataset": dataset,
        "optimizer": "SGD",
        "random_perms": True,
        "compound_formatting": "logical_ordering"
    }
)

while n <= iterations:
    hprev = torch.zeros(hidden_size,)  # Reset RNN memory at the start of each iteration

    # Get the current window of input data
    window = training_inputs[p]
    target = training_outputs[p]

    # Apply padding with the last value to the window (if needed)
    window = pad_with_last_value(window, max_race_size).to(device)
    target = pad_with_last_value(target, max_race_size).to(device)

    # Calculate loss and backpropagate
    loss, hprev = calculate_loss(model, window, target, hprev, max_race_size)

    optim.zero_grad()  # Clear previous gradients
    loss.backward()    # Backpropagate the loss

    optim.step()  # Update model parameters

    # Logging every 500 iterations
    if n % 500 == 0:
        print(f'[{n}] Loss: {loss.item():.2f}')

        # Evaluate and log metrics
        testing_accuracy = evaluate_model(testing_inputs, testing_outputs)
        training_accuracy = evaluate_model(training_inputs, training_outputs)

        wandb.log({
            "iteration": n,
            "loss": loss.item(),
            "training_accuracy": training_accuracy,
            "testing_accuracy": testing_accuracy,
        })

        # Save the model
        model_path = "model.pth"
        torch.save(model.state_dict(), model_path)

        # Log the model artifact to WandB
        artifact = wandb.Artifact("basic-rnn-binary-classifier", type="model")
        artifact.add_file(model_path)
        wandb.log_artifact(artifact)

    n += 1  # Increment the iteration counter
    p += 1  # Increment the window counter

    # If we reach the end of the training dataset, shuffle and restart
    if len(training_inputs) == p:
        p = 0  # Restart from the beginning
        permutation = torch.randperm(len(training_inputs))
        training_inputs = [training_inputs[i] for i in permutation]
        training_outputs = [training_outputs[i] for i in permutation]

<ipython-input-39-81c9b9439275>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  window = torch.tensor(window)  # Convert list to tensor if window is a list
<ipython-input-39-81c9b9439275>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)  # Convert list to tensor if target is a list


TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not list